In [1]:
pip install rouge-score

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/133.7 kB ? eta -:--:--
   --------- ------------------------------ 30.7/133.7 kB 1.3 MB/s eta 0:00:01
   --------- ------------------------------ 30.7/133.7 kB 1.3 MB/s eta 0:00:01
   -------------------- ------------------ 71.7/133.7 kB 563.7 kB/s eta 0:00:01
   -------------------- ------------------ 71.7/133.7 kB 563.7 kB/s eta 0:00:01
   -------------------------------- ----- 112.6/133.7 kB 504.4 kB/s eta 0:00:01
   -------------------------------------- 133.7/133.7 kB 527.6 kB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24972 sha256=bcfc58b61e3da89be6bf212e51


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from rouge_score import rouge_scorer

In [3]:
# Load PEGASUS model and tokenizer
model_name = "google/pegasus-xsum"
model = PegasusForConditionalGeneration.from_pretrained(model_name)
tokenizer = PegasusTokenizer.from_pretrained(model_name)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

In [4]:
# Sample text
text = """
moreover the central bank initiated measures to prioritise essential imports and restrict capital outflows through appropriate control measures while continuing the requirement for the mandatory sale of foreign exchange to the central bank by licensed banks based on the conversion of repatriated foreign exchange and commenced providing daily guidance on the exchange rate to curtail undue intraday volatilities.moreover the central bank continuous financial sector oversight and adoption of appropriate regulatory measures along with effective communication ensured financial system stability amidst severe socio economic distress.economic price and financial system stability outlook and policieseconomy.the credibility of the central bank is highly related to the independence of the bank.in this regard the envisaged enactment of the new central bank of sri lanka act will contribute immensely to improving the independence and credibility of the central bank which in turn will support the current disinflation episode and further strengthen the anchoring of inflation expectations.globalization and global disinflation federal reserve bank of kansas city conference on monetary policy and uncertainty adapting to changing economy.economic price and financial system stability outlook and policiesyears the surge in imported prices as well as increases in the cost of non food categories such as restaurants and hotels health and education etc.this continued reduction in core inflation was attributed to the strong policy measures taken by the central bank to address the build up of demand driven inflationary pressures and adverse inflation expectations.the government and the central bank initiated measures to limit foreign exchange outflows while taking initiatives to promote inflows. import demand was reduced notably reflecting the impact of significantly tightened monetary policy and subdued demand conditions.the central bank played major role in managing foreign exchange to ensure the supply of essential goods and services under extremely challenging circumstances during.figure balance of paymentssource central bank of sri lankatrade balance current account balance overall balance us billion
"""

In [10]:
# Reference summary (for ROUGE evaluation)
reference_summary = """
The central bank prioritized essential imports, restricted capital outflows, and mandated the sale of foreign exchange to ensure financial stability. Measures were taken to limit foreign exchange outflows and promote inflows. Tightened monetary policy reduced import demand. The envisaged enactment of the new Central Bank of Sri Lanka Act is expected to improve the bank's independence and credibility, supporting disinflation and inflation expectations anchoring.
"""

In [6]:
# Preprocess text for PEGASUS
inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)


In [8]:
# Generate summary
summary_ids = model.generate(inputs, num_beams=4, min_length=50, max_length=100, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)

The central bank took a number of policy measures to address the build up of demand driven inflationary pressures and adverse inflation expectations in the first half of the year and continued to do so in the second half of the year to support economic growth and financial system stability.


In [11]:
# Calculate ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_summary, summary)

print("\nROUGE Scores:")
print(f"ROUGE-1: {scores['rouge1']}")
print(f"ROUGE-2: {scores['rouge2']}")
print(f"ROUGE-L: {scores['rougeL']}")


ROUGE Scores:
ROUGE-1: Score(precision=0.48, recall=0.36923076923076925, fmeasure=0.4173913043478261)
ROUGE-2: Score(precision=0.08163265306122448, recall=0.0625, fmeasure=0.07079646017699115)
ROUGE-L: Score(precision=0.28, recall=0.2153846153846154, fmeasure=0.24347826086956526)
